# Imports

In [1]:
%load_ext sql
%sql sqlite:///TestQ.db

import pandas as pd
import sqlite3
# Connect to DB
conn = sqlite3.connect("TestQ.db")

https://github.com/yashjadwani/15-Days-SQL-challenge-by-PrepVector

# Question 1

In [2]:
# -- Question

# You’re given two tables: users and events. 
# The events table holds values of all of the user events in the action column (‘like’, ‘comment’, or ‘post’).

# Write a query to get the percentage of users that have never liked or commented, rounded to two decimal places.

In [3]:
%%sql

CREATE TABLE users (
user_id INT PRIMARY KEY,
name VARCHAR(50)
);

INSERT INTO users (user_id, name) VALUES
(1, 'John'),
(2, 'Jane'),
(3, 'Bob'),
(4, 'Alice'),
(5, 'Mike'),
(6, 'ABC');

CREATE TABLE events (
event_id INT PRIMARY KEY,
user_id INT,
action VARCHAR(10),
created_at DATETIME,
FOREIGN KEY (user_id) REFERENCES users(user_id)
);

INSERT INTO events (event_id, user_id, action, created_at) VALUES
(1, 1, 'post', '2024-01-01 10:00:00'),
(2, 1, 'post', '2024-01-01 14:00:00'),
(3, 1, 'post', '2024-01-02 09:00:00'),
(4, 2, 'like', '2024-01-01 10:05:00'),
(5, 2, 'comment', '2024-01-01 10:10:00'),
(6, 2, 'post', '2024-01-01 15:00:00'),
(7, 2, 'like', '2024-01-02 10:00:00'),
(8, 2, 'comment', '2024-01-02 10:30:00'),
(9, 3, 'post', '2024-01-01 11:00:00'),
(10, 3, 'post', '2024-01-02 13:00:00'),
(11, 3, 'post', '2024-01-03 09:00:00'),
(12, 4, 'post', '2024-01-02 09:00:00'),
(13, 4, 'post', '2024-01-02 16:00:00'),
(14, 4, 'post', '2024-01-03 11:00:00'),
(15, 5, 'like', '2024-01-02 14:00:00'),
(16, 5, 'post', '2024-01-03 10:00:00'),
(17, 5, 'like', '2024-01-03 15:00:00');

 * sqlite:///TestQ.db
Done.
6 rows affected.
Done.
17 rows affected.


[]

In [27]:
query = """
WITH user_actions as (
select user_id, count(*) as num_like_comments from events 
where action != 'post'
group by 1)

SELECT 
    ROUND(
    COUNT(DISTINCT CASE WHEN e.num_like_comments is NULL THEN u.user_id END) * 100.0 / COUNT(DISTINCT u.user_id) 
    , 2) as perc_inactive
FROM 
    users u 
LEFT JOIN 
    user_actions e ON u.user_id = e.user_id;
"""

df = pd.read_sql_query(query, conn)
df

,perc_inactive
0,66.67


# Question 2

In [ ]:
# -- Question 

# Given a table of transactions and a table of users
# write a query to determine if users tend to order more to their primary address versus other addresses.

# Note: Return the percentage of transactions ordered to their home address as home_address_percent.

In [35]:
%%sql

DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS  transactions;

 * sqlite:///TestQ.db
Done.
Done.


[]

In [36]:
%%sql

CREATE TABLE transactions (
id INT PRIMARY KEY,
user_id INT,
created_at DATETIME,
shipping_address VARCHAR(255)
);

-- Transactions sample data
INSERT INTO transactions (id, user_id, created_at, shipping_address) VALUES
(1, 1, '2025-01-15 10:30:00', '123 Main St'), 
(2, 1, '2025-01-16 11:45:00', '789 Oak Ave'), 
(3, 2, '2025-01-17 14:20:00', '456 Elm St'), 
(4, 2, '2025-01-18 15:10:00', '123 Pine Rd'), 
(5, 3, '2025-01-19 16:05:00', '789 Oak Ave'), 
(6, 3, '2025-01-20 17:40:00', '123 Main St'),
(7, 3, '2025-01-21 17:45:00', '123 Main St');

CREATE TABLE users (
id INT PRIMARY KEY,
name VARCHAR(255),
address VARCHAR(255)
);

-- Users sample data
INSERT INTO users (id, name, address) VALUES
(1, 'John Doe', '123 Main St'),
(2, 'Jane Smith', '456 Elm St'),
(3, 'Alice Johnson', '789 Oak Ave');

 * sqlite:///TestQ.db
Done.
Done.
Done.
Done.


[]

In [42]:
query = """
select
    u.id,
    u.name, 
    count(distinct case when u.address = t.shipping_address then t.id end) as home_address_orders,
    count(distinct t.id) as total_orders,
    ROUND(count(distinct case when u.address = t.shipping_address then t.id end) * 100.0 / count(distinct t.id), 2) as home_address_percent
from
    users u left join transactions t on u.id = t.user_id
group by
    1, 2;
"""

df = pd.read_sql_query(query, conn)
df

,id,name,home_address_orders,total_orders,home_address_percent
0,1,John Doe,1,2,50.00
1,2,Jane Smith,1,2,50.00
2,3,Alice Johnson,1,3,33.33


In [46]:
query = """
WITH agg_transactions as(
select user_id, shipping_address, count(*) as orders from transactions group by 1, 2
)

select
    u.id,
    u.name, 
    sum(case when u.address = t.shipping_address then orders else 0 end) as home_address_orders,
    sum(orders) as total_orders,
    ROUND(sum(case when u.address = t.shipping_address then orders else 0 end) * 100.0 / sum(orders), 2) as home_address_percent
from
    users u left join agg_transactions t on u.id = t.user_id
group by
    1, 2;
"""

df = pd.read_sql_query(query, conn)
df

,id,name,home_address_orders,total_orders,home_address_percent
0,1,John Doe,1,2,50.00
1,2,Jane Smith,1,2,50.00
2,3,Alice Johnson,1,3,33.33


# Question 3

In [ ]:
# -- Question

# Given a table of job postings, 
# write a query to retrieve the number of users 
# that have posted each job only once and the number of users that have posted at least one job multiple times.

In [47]:
%%sql

CREATE TABLE job_postings (
    id INT PRIMARY KEY,
    user_id INT,
    job_id INT,
    posted_date DATETIME
);

INSERT INTO job_postings (id, user_id, job_id, posted_date) VALUES
    (1, 1, 101, '2024-01-01'),
    (2, 1, 102, '2024-01-02'),
    (3, 2, 201, '2024-01-01'),
    (4, 2, 201, '2024-01-15'),
    (5, 2, 202, '2024-01-03'),
    (6, 3, 301, '2024-01-01'),
    (7, 4, 401, '2024-01-01'),
    (8, 4, 401, '2024-01-15'),
    (9, 4, 402, '2024-01-02'),
    (10, 4, 402, '2024-01-16'),
    (11, 5, 501, '2024-01-05'),
    (12, 5, 502, '2024-01-10');

 * sqlite:///TestQ.db
Done.
12 rows affected.


[]

In [ ]:
# id, user_id, job_id, posted_date

In [50]:
query = """
with agg_jobs as (
select user_id, job_id, count(*) as num_posts from job_postings group by 1, 2
)

select
    user_id,
    max(num_posts) as max_num_posts,
    case when max(num_posts) = 1 then 'Single Poster' else 'Multiple Poster' end as class
from
    agg_jobs
group by 1
"""

df = pd.read_sql_query(query, conn)
df

,user_id,max_num_posts,class
0,1,1,Single Poster
1,2,2,Multiple Poster
2,3,1,Single Poster
3,4,2,Multiple Poster
4,5,1,Single Poster


In [58]:
query = """
with agg_jobs as (
select user_id, job_id, count(*) as num_posts from job_postings group by 1, 2
),

agg_jobs_max as (
select user_id, max(num_posts) as max_num_posts from agg_jobs group by 1
)

select
    sum(case when max_num_posts = 1 then 1 else 0 end) as num_single_posters,
    sum(case when max_num_posts > 1 then 1 else 0 end) as num_multiple_posters
from
    agg_jobs_max
"""

df = pd.read_sql_query(query, conn)
df

,num_single_posters,num_multiple_posters
0,3,2


# Question 4

In [ ]:
# -- Question

# Given a table of customer sales in a retail store 
# with columns id, transaction_value, and created_at representing the date and time for each transaction, 
# write a query to get the last transaction for each day.

# The output should include the ID of the transaction, datetime of the transaction, and the transaction amount. 
# Order the transactions by datetime.


In [59]:
%%sql

CREATE TABLE customer_sales (
id INT PRIMARY KEY,
transaction_value DECIMAL(10, 2),
created_at DATETIME
);

INSERT INTO customer_sales (id, transaction_value, created_at)
VALUES
(1, 50.00, '2025-01-23 10:15:00'),
(2, 30.00, '2025-01-23 15:45:00'),
(3, 20.00, '2025-01-23 18:30:00'),
(4, 45.00, '2025-01-24 09:20:00'),
(5, 60.00, '2025-01-24 22:10:00'),
(6, 25.00, '2025-01-25 11:30:00'),
(7, 35.00, '2025-01-25 14:50:00'),
(8, 55.00, '2025-01-25 19:05:00');


 * sqlite:///TestQ.db
Done.
8 rows affected.


[]

In [75]:
query = """
with ranks as (
select 
    *, 
    rank() over(partition by date(created_at) order by created_at desc) as rank
from
    customer_sales
)

select id, created_at, transaction_value from ranks where rank = 1 ORDER BY created_at asc;
"""

df = pd.read_sql_query(query, conn)
df

,id,created_at,transaction_value
0,3,2025-01-23 18:30:00,20
1,5,2025-01-24 22:10:00,60
2,8,2025-01-25 19:05:00,55


# Question 5

In [ ]:
# -- Question

# Consider the events table, which contains information about the phases of writing a new social media post.

# The action column can have values post_enter, post_submit, or post_canceled 
# for when a user starts to write (post_enter), ends up canceling their post (post_cancel), or posts it (post_submit). 
# Write a query to get the post-success rate for each day in the month of January 2020.

# Note: Post Success Rate is defined as 
# the number of posts submitted (post_submit) divided by the number of posts entered (post_enter) for each day.


In [78]:
%%sql

drop table if exists events;

CREATE TABLE events (
user_id INT,
created_at DATETIME,
action VARCHAR(20)
);

INSERT INTO events VALUES
(1, '2020-01-01 10:00:00', 'post_enter'),
(1, '2020-01-01 10:05:00', 'post_submit'),
(2, '2020-01-01 11:00:00', 'post_enter'),
(2, '2020-01-01 11:10:00', 'post_canceled'),
(3, '2020-01-01 15:00:00', 'post_enter'),
(3, '2020-01-01 15:30:00', 'post_submit'),
(4, '2020-01-02 09:00:00', 'post_enter'),
(4, '2020-01-02 09:15:00', 'post_canceled'),
(5, '2020-01-02 10:00:00', 'post_enter'),
(5, '2020-01-02 10:10:00', 'post_canceled'),
(10, '2020-01-15 14:00:00', 'post_enter'),
(10, '2020-01-15 14:30:00', 'post_submit'),
(6, '2019-12-31 23:55:00', 'post_enter'),
(6, '2020-01-01 00:05:00', 'post_submit'),
(7, '2020-02-01 00:00:00', 'post_enter'),
(7, '2020-02-01 00:10:00', 'post_submit'),
(8, '2019-01-15 10:00:00', 'post_enter'),
(8, '2019-01-15 10:30:00', 'post_submit'),
(9, '2021-01-01 09:00:00', 'post_enter'),
(9, '2021-01-01 09:10:00', 'post_canceled');

 * sqlite:///TestQ.db
Done.
Done.
20 rows affected.


[]

In [92]:
query = """
select 
    date(created_at) as date,
    strftime('%Y-%m', created_at) as year_month,
    sum(case when action = 'post_submit' then 1 else 0 end) * 100.0/ sum(case when action = 'post_enter' then 1 else 0 end) as post_success_rate
from 
    events
where 
    strftime('%Y-%m', created_at) = '2020-01'
group by date
order by date;
"""

df = pd.read_sql_query(query, conn)
df

,date,year_month,post_success_rate
0,2020-01-01,2020-01,100.0
1,2020-01-02,2020-01,0.0
2,2020-01-15,2020-01,100.0


# Question 6

In [ ]:
# -- Question

# Given a table of transactions and products, 
# write a query to return the 
# product ID, product price, and average transaction price of all products with a price greater than the average transaction price.

In [94]:
%%sql

drop table if exists products;
drop table if exists transactions;

CREATE TABLE products (
product_id INT PRIMARY KEY,
price DECIMAL(10,2)
);

INSERT INTO products (product_id, price) VALUES
(1, 100.00),
(2, 150.00),
(3, 75.00),
(4, 200.00),
(5, 120.00);

CREATE TABLE transactions (
transaction_id INT PRIMARY KEY,
product_id INT,
amount DECIMAL(10,2),
FOREIGN KEY (product_id) REFERENCES products(product_id)
);

INSERT INTO transactions (transaction_id, product_id, amount) VALUES
(1, 1, 95.00),
(2, 1, 98.00),
(3, 2, 145.00),
(4, 2, 150.00),
(5, 3, 70.00),
(6, 4, 190.00),
(7, 4, 195.00),
(8, 5, 115.00);

 * sqlite:///TestQ.db
Done.
Done.
Done.
5 rows affected.
Done.
8 rows affected.


[]

In [96]:
query = """
with avg_transaction_price as (
select product_id, avg(amount) as avg_price from transactions group by 1
)

select 
    p.product_id, 
    p.price,
    t.avg_price
from 
    products p left join avg_transaction_price t on p.product_id =  t.product_id
where
    p.price > t.avg_price;
"""

df = pd.read_sql_query(query, conn)
df

,product_id,price,avg_price
0,1,100,96.5
1,2,150,147.5
2,3,75,70.0
3,4,200,192.5
4,5,120,115.0


# Question 7

In [97]:
# -- Question

# We’re given a table of product purchases. Each row in the table represents an individual user product purchase.

# Write a query to get the number of customers that were upsold by purchasing additional products.

# Note: If a customer purchased multiple products on the same day, it does not count as an upsell. 
# An upsell is considered only if they made purchases on separate days


In [99]:
%%sql

drop table if exists transactions;

CREATE TABLE transactions (
id INTEGER PRIMARY KEY,
user_id INTEGER,
created_at DATETIME,
product_id INTEGER,
quantity INTEGER
);

INSERT INTO transactions (id, user_id, created_at, product_id, quantity) VALUES
(1, 101, '2024-01-01 10:00:00', 1, 1),  
(2, 101, '2024-01-01 14:00:00', 2, 1),
(3, 101, '2024-01-15 09:00:00', 3, 1), 
(4, 102, '2024-01-05 11:00:00', 1, 2),
(5, 102, '2024-01-05 11:30:00', 2, 1),
(6, 103, '2024-01-02 15:00:00', 1, 1),
(7, 104, '2024-01-01 09:00:00', 1, 1),
(8, 104, '2024-01-02 10:00:00', 2, 1),
(9, 104, '2024-01-03 11:00:00', 3, 1);


 * sqlite:///TestQ.db
Done.
Done.
9 rows affected.


[]

In [109]:
query = """
WITH 
cust_table_pre AS (
    SELECT *, DATE(created_at) AS txn_date
    FROM transactions 
),

cust_table AS (
    SELECT 
        user_id, 
        COUNT(DISTINCT txn_date || ',' || product_id) AS num_unique_product_days
    FROM cust_table_pre 
    GROUP BY user_id
)

SELECT * FROM cust_table;
"""

df = pd.read_sql_query(query, conn)
df

,user_id,num_unique_product_days
0,101,3
1,102,2
2,103,1
3,104,3


# Question 8

In [ ]:
# -- Question 

# Write a query to return pairs of projects where the end date of one project matches the start date of another project.

In [2]:
%%sql

CREATE TABLE projects (
id INTEGER PRIMARY KEY,
title VARCHAR(100),
start_date DATETIME,
end_date DATETIME,
budget FLOAT
);

INSERT INTO projects (id, title, start_date, end_date, budget) VALUES
(1, 'Website Redesign', '2024-01-01', '2024-02-15', 50000),
(2, 'Mobile App Phase 1', '2024-02-15', '2024-04-01', 75000),
(3, 'Database Migration', '2024-04-01', '2024-05-15', 60000),
(4, 'Cloud Integration', '2024-03-01', '2024-04-15', 45000),
(5, 'Security Audit', '2024-05-15', '2024-06-30', 30000);

 * sqlite:///TestQ.db
Done.
5 rows affected.


[]

In [4]:
query = """
select 
    a.id as project1_id,
    a.title as project1_title,
    b.id as project2_id, 
    b.title as project2_title
from
    projects a inner join projects b on a.start_date = b.end_date;
"""
df = pd.read_sql_query(query, conn)
df

,project1_id,project1_title,project2_id,project2_title
0,2,Mobile App Phase 1,1,Website Redesign
1,3,Database Migration,2,Mobile App Phase 1
2,5,Security Audit,3,Database Migration


In [ ]:
# This is better answer to the question
query = """
select 
    a.id as project1_id,
    a.title as project1_title,
    b.id as project2_id, 
    b.title as project2_title
from
    projects a inner join projects b on a.end_date = b.start_date;
"""
df = pd.read_sql_query(query, conn)
df

,project1_id,project1_title,project2_id,project2_title
0,1,Website Redesign,2,Mobile App Phase 1
1,2,Mobile App Phase 1,3,Database Migration
2,3,Database Migration,5,Security Audit


In [ ]:
# Edge Cases

# Same project matching itself? You're not excluding that — is it allowed?. This is possible when a project lasts only one day
# If not, add: a.id != b.id

# Multiple projects with same end/start dates — are duplicates expected and acceptable?

# Question 9

In [ ]:
# -- Question

# Given a table containing data for monthly sales, 
# write a query to find the total amount of each product sold for each month, with each product as its own column in the output table.

In [6]:
%%sql

CREATE TABLE monthly_sales (
month DATE,
product_id INTEGER,
amount_sold INTEGER
);

INSERT INTO monthly_sales (month, product_id, amount_sold) VALUES
('2021-01-01', 1, 100),
('2021-01-01', 2, 300),
('2021-02-01', 1, 150),
('2021-02-01', 1, 50),
('2021-02-01', 2, 250),
('2021-03-01', 1, 120),
('2021-03-01', 4, 250),
('2021-04-01', 2, -30),
('2021-04-01', 3, 200),
('2021-05-01', 3, 175),
('2021-06-01', 1, 0),
('2021-06-01', 2, 100);

 * sqlite:///TestQ.db
Done.
12 rows affected.


[]

In [10]:
query = """
select 
    month,
    sum(case when product_id = 1 then amount_sold else 0 end) as product_id_1_sales,
    sum(case when product_id = 2 then amount_sold else 0 end) as product_id_2_sales,
    sum(case when product_id = 3 then amount_sold else 0 end) as product_id_3_sales,
    sum(case when product_id = 4 then amount_sold else 0 end) as product_id_3_sales
from
    monthly_sales
group by 
    month;
"""
df = pd.read_sql_query(query, conn)
df

,month,product_id_1_sales,product_id_2_sales,product_id_3_sales,product_id_3_sales
0,2021-01-01,100,300,0,0
1,2021-02-01,200,250,0,0
2,2021-03-01,120,0,0,250
3,2021-04-01,0,-30,200,0
4,2021-05-01,0,0,175,0
5,2021-06-01,0,100,0,0


In [ ]:
# This is scalable as long as: The number of products is small and known ahead of time.

# If not, then use pivot if your sql supports it

# Question 10

In [ ]:
# -- Question

# A dating website’s schema is represented by a table of people that like other people. The table has three columns. 
# One column is the user_id, 
# another column is the liker_id which is the user_id of the user doing the liking, 
# and the last column is the date time that the like occurred.

# Write a query to count the number of liker’s likers (the users that like the likers) if the liker has one.

In [16]:
%%sql

drop table if exists likes;

CREATE TABLE likes (
user_id VARCHAR(50),
created_at DATETIME,
liker_id VARCHAR(50)
);

INSERT INTO likes (user_id, created_at, liker_id) VALUES
('A', '2024-01-01 10:00:00', 'B'),
('B', '2024-01-01 11:00:00', 'A'),
('B', '2024-01-01 11:00:00', 'C'),
('B', '2024-01-01 12:00:00', 'D'),
('B', '2024-01-01 13:00:00', 'E'),
('C', '2024-01-02 10:00:00', 'A'),
('D', '2024-01-02 14:00:00', 'E'),
('E', '2024-01-02 15:00:00', 'F'),
('B', '2024-01-03 09:00:00', 'G'),
('H', '2024-01-03 10:00:00', 'A'),
('B', '2024-01-03 11:00:00', 'C'),
('I', '2024-01-03 12:00:00', 'I');

 * sqlite:///TestQ.db
Done.
Done.
12 rows affected.


[]

In [20]:
query = """
select
    a.user_id,
    a.liker_id,
    b.user_id as liker_user,
    b.liker_id as likers_liker
from
    likes a
inner join
    likes b 
on
    a.liker_id = b.user_id and a.user_id = b.liker_id
where
    a.user_id != a.liker_id
"""

df = pd.read_sql_query(query, conn)
df

,user_id,liker_id,liker_user,likers_liker
0,A,B,B,A
1,B,A,A,B


In [ ]:
# Write a query to count the number of liker’s likers (the users that like their own likers)

query = """
with likers_likers as (
select
    a.user_id,
    a.liker_id,
    b.user_id as liker_user,
    b.liker_id as likers_liker
from
    likes a
inner join
    likes b 
on
    a.liker_id = b.user_id and a.user_id = b.liker_id
where
    a.user_id != a.liker_id )

select count(distinct likers_liker) as num_likers_likers from likers_likers
"""

df = pd.read_sql_query(query, conn)
df

,num_likers_likers
0,2


In [ ]:
# Write a query to count the number of liker’s likers (the users that like the likers)

query = """
select
    a.user_id,
    a.liker_id,
    b.user_id as liker_user,
    b.liker_id as likers_liker
from
    likes a
inner join
    likes b 
on 
    a.liker_id = b.user_id
"""

df = pd.read_sql_query(query, conn)
df

,user_id,liker_id,liker_user,likers_liker
0,A,B,B,A
1,A,B,B,C
2,A,B,B,C
3,A,B,B,D
4,A,B,B,E
5,A,B,B,G
6,B,A,A,B
7,B,C,C,A
8,B,D,D,E
9,B,E,E,F


In [ ]:
query = """
with likers_likers as (
  select
    a.user_id,
    a.liker_id,
    b.user_id as liker_user,
    b.liker_id as likers_liker
  from
    likes a
  inner join
    likes b 
    on a.liker_id = b.user_id
)

select 
  count(distinct likers_liker) as num_likers_likers
from 
  likers_likers
"""

df = pd.read_sql_query(query, conn)
df

,num_likers_likers
0,8


# Question 11

In [ ]:
# -- Question

# Given a table of song_plays and a table of users, 
# write a query to extract the earliest date each user played their third unique song and order by date played.

In [25]:
%%sql

drop table if exists users;
drop table if exists song_plays;

CREATE TABLE users (
id INTEGER PRIMARY KEY,
username VARCHAR(50)
);

INSERT INTO users (id, username) VALUES
(1, 'john_doe'),
(2, 'jane_smith'),
(3, 'bob_wilson');

CREATE TABLE song_plays (
id INTEGER PRIMARY KEY,
played_at DATETIME,
user_id INTEGER,
song_id INTEGER
);

INSERT INTO song_plays (id, played_at, user_id, song_id) VALUES
(1, '2024-01-01 10:00:00', 1, 101),
(2, '2024-01-01 14:00:00', 1, 101),
(3, '2024-01-02 09:00:00', 1, 102),
(4, '2024-01-03 16:00:00', 1, 103),
(5, '2024-01-04 11:00:00', 1, 104),
(6, '2024-01-01 09:00:00', 2, 201),
(7, '2024-01-01 15:00:00', 2, 202),
(8, '2024-01-02 10:00:00', 2, 203),
(9, '2024-01-02 14:00:00', 2, 203),
(10, '2024-01-01 12:00:00', 3, 301),
(11, '2024-01-02 13:00:00', 3, 302);

 * sqlite:///TestQ.db
Done.
Done.
Done.
3 rows affected.
Done.
11 rows affected.


[]

In [37]:
query = """
with unique_songs as (
select 
    user_id, 
    song_id, 
    min(played_at) as earliest_play 
from
    song_plays
group by 
    user_id, song_id
),


ranked_songs as (
select
    *,
    rank() over(partition by user_id order by earliest_play asc) as song_rank
from
    unique_songs
)


select
    b.username,
    a.*
from
    ranked_songs a left join users b
on
    a.user_id = b.id
where
    song_rank = 3
"""

df = pd.read_sql_query(query, conn)
df

,username,user_id,song_id,earliest_play,song_rank
0,john_doe,1,103,2024-01-03 16:00:00,3
1,jane_smith,2,203,2024-01-02 10:00:00,3


In [ ]:
# ❗ Minor Issue: rank() vs row_number()
# In your use case:

# Each user-song pair is already distinct by earliest_play.

# So you can use row_number() instead of rank() — this avoids any unexpected duplicate rank = 3 entries in case of ties.

# If there can be ties in played_at, 
# rank() may return multiple rows per user — which might not match the question's requirement of the third unique song.

# Question 12

In [40]:
# -- Question

# Given a table with event logs, find the top five users with the longest continuous streak of visiting the platform in 2020.

# Note: A continuous streak counts if the user visits the platform at least once per day on consecutive days.

In [39]:
%%sql

DROP TABLE IF EXISTS events;

CREATE TABLE events (
user_id INT,
created_at DATETIME,
url VARCHAR(255)
);

INSERT INTO events (user_id, created_at, url) VALUES

(1, '2019-12-30 10:00:00', 'https://example.com/2019-page1'),
(1, '2019-12-31 11:00:00', 'https://example.com/2019-page2'),
(2, '2019-11-15 12:00:00', 'https://example.com/2019-profile1'),
(2, '2019-11-16 13:00:00', 'https://example.com/2019-profile2'),
(3, '2019-10-20 14:00:00', 'https://example.com/2019-blog1'),
(4, '2019-09-25 16:00:00', 'https://example.com/2019-review1'),
(4, '2019-09-26 17:00:00', 'https://example.com/2019-review2'),
(5, '2019-08-30 18:00:00', 'https://example.com/2019-summer1'),
(5, '2019-08-31 19:00:00', 'https://example.com/2019-summer2'),
(6, '2019-07-15 20:00:00', 'https://example.com/2019-page1'),
(6, '2019-07-16 21:00:00', 'https://example.com/2019-page2'),
(1, '2020-01-01 10:00:00', 'https://example.com/page1'),
(1, '2020-01-02 11:00:00', 'https://example.com/page2'),
(1, '2020-01-04 12:00:00', 'https://example.com/page3'),
(1, '2020-01-05 13:00:00', 'https://example.com/page4'),
(1, '2020-01-06 14:00:00', 'https://example.com/page5'),
(1, '2020-01-07 12:00:00', 'https://example.com/page7'),
(1, '2020-01-08 12:00:00', 'https://example.com/page8'),
(1, '2020-01-09 12:00:00', 'https://example.com/page9'),
(1, '2020-01-10 12:00:00', 'https://example.com/page10'),
(2, '2020-02-10 15:00:00', 'https://example.com/dashboard'),
(2, '2020-02-11 16:00:00', 'https://example.com/profile'),
(2, '2020-02-12 17:00:00', 'https://example.com/settings'),
(2, '2020-02-14 18:00:00', 'https://example.com/messages'),
(2, '2020-02-15 19:00:00', 'https://example.com/notifications'),
(2, '2020-02-16 20:00:00', 'https://example.com/search'),
(2, '2020-02-17 21:00:00', 'https://example.com/help'),
(2, '2020-02-18 21:00:00', 'https://example.com/help2'),
(3, '2020-03-15 22:00:00', 'https://example.com/blog'),
(3, '2020-05-20 01:00:00', 'https://example.com/products'),
(3, '2020-05-22 02:00:00', 'https://example.com/cart'),
(4, '2020-12-28 04:00:00', 'https://example.com/year-end'),
(4, '2020-12-30 05:00:00', 'https://example.com/review'),
(4, '2020-12-31 06:00:00', 'https://example.com/summary'),
(5, '2020-04-01 08:00:00', 'https://example.com/spring1'),
(5, '2020-04-02 09:00:00', 'https://example.com/spring2'),
(5, '2020-07-15 10:00:00', 'https://example.com/summer1'),
(5, '2020-07-17 11:00:00', 'https://example.com/summer2'),
(5, '2020-07-18 12:00:00', 'https://example.com/summer3'),
(5, '2020-10-20 13:00:00', 'https://example.com/autumn1'),
(6, '2020-06-01 15:00:00', 'https://example.com/page1'),
(6, '2020-06-02 16:00:00', 'https://example.com/page2'),
(6, '2020-06-03 17:00:00', 'https://example.com/page3'),
(6, '2020-06-04 18:00:00', 'https://example.com/page4'),
(6, '2020-06-05 19:00:00', 'https://example.com/page5'),
(6, '2020-06-06 20:00:00', 'https://example.com/page6'),
(6, '2020-06-07 21:00:00', 'https://example.com/page7'),
(6, '2020-06-08 22:00:00', 'https://example.com/page8'),
(6, '2020-06-09 23:00:00', 'https://example.com/page9'),
(6, '2020-06-10 00:00:00', 'https://example.com/page10'),
(7, '2020-08-01 01:00:00', 'https://example.com/august1'),
(7, '2020-08-02 02:00:00', 'https://example.com/august2'),
(7, '2020-08-03 03:00:00', 'https://example.com/august3'),
(7, '2020-08-04 03:00:00', 'https://example.com/august6'),
(8, '2020-09-11 05:00:00', 'https://example.com/september2'),
(9, '2020-11-15 06:00:00', 'https://example.com/november1'),
(9, '2020-11-16 07:00:00', 'https://example.com/november2'),
(9, '2020-11-17 08:00:00', 'https://example.com/november3'),
(10, '2020-05-01 09:00:00', 'https://example.com/may1'),
(10, '2020-05-02 10:00:00', 'https://example.com/may2'),
(1, '2021-01-01 22:00:00', 'https://example.com/2021-page1'),
(1, '2021-01-02 23:00:00', 'https://example.com/2021-page2'),
(2, '2021-02-10 00:00:00', 'https://example.com/2021-profile1'),
(2, '2021-02-11 01:00:00', 'https://example.com/2021-profile2'),
(3, '2021-03-15 02:00:00', 'https://example.com/2021-blog1'),
(3, '2021-03-16 03:00:00', 'https://example.com/2021-blog2'),
(4, '2021-04-20 04:00:00', 'https://example.com/2021-review1'),
(4, '2021-04-21 05:00:00', 'https://example.com/2021-review2'),
(5, '2021-05-25 06:00:00', 'https://example.com/2021-summer1'),
(5, '2021-05-26 07:00:00', 'https://example.com/2021-summer2'),
(6, '2021-06-30 08:00:00', 'https://example.com/2021-page1'),
(6, '2021-07-01 09:00:00', 'https://example.com/2021-page2'),
(11, '2020-01-15 10:00:00', 'https://example.com/complex1'),
(11, '2020-01-16 11:00:00', 'https://example.com/complex2'),
(11, '2020-01-18 12:00:00', 'https://example.com/complex3'),
(11, '2020-01-19 13:00:00', 'https://example.com/complex4'),
(12, '2020-04-10 14:00:00', 'https://example.com/scenario1'),
(12, '2020-04-11 15:00:00', 'https://example.com/scenario2'),
(12, '2020-04-13 16:00:00', 'https://example.com/scenario3'),
(12, '2020-04-14 17:00:00', 'https://example.com/scenario4');

 * sqlite:///TestQ.db
Done.
Done.
80 rows affected.


[]

In [ ]:
# (user_id, created_at, url)

In [ ]:
query = """
with daily_logs as (
select
    distinct
    user_id,
    date(created_at) as visit_date,
    row_number() over(partition by user_id order by date(created_at)) as rn
from
    events
)

select
    user_id,
    visit_date,
    julianday(visit_date) - rn as streak_group
from 
    daily_logs
"""

# date_diff(visit_date, rn, day) -> this will also give the streak group

df = pd.read_sql_query(query, conn)
df

,user_id,visit_date,streak_group
0,1,2019-12-30,2458846.5
1,1,2019-12-31,2458846.5
2,1,2020-01-01,2458846.5
3,1,2020-01-02,2458846.5
4,1,2020-01-04,2458847.5
...,...,...,...
75,11,2020-01-19,2458863.5
76,12,2020-04-10,2458948.5
77,12,2020-04-11,2458948.5
78,12,2020-04-13,2458949.5


In [57]:
df.query("user_id == 1")

,user_id,visit_date,streak_group
0,1,2019-12-30,2458846.5
1,1,2019-12-31,2458846.5
2,1,2020-01-01,2458846.5
3,1,2020-01-02,2458846.5
4,1,2020-01-04,2458847.5
5,1,2020-01-05,2458847.5
6,1,2020-01-06,2458847.5
7,1,2020-01-07,2458847.5
8,1,2020-01-08,2458847.5
9,1,2020-01-09,2458847.5
